# RoWer usage  

## Purpose of RoWer  
The ecoinvent database contains geographies (i.e. locations) called Rest-of_World, or RoW for short. These are automatically generated by the ecoinvent service layer when there are datasets for an activity in specific regions, and when these regions don't span the whole world.  
RoW are essentially "the globe (GLO) less locations for which there are datasets". 
Because coverage of different activities in the database is unequal, the geography "RoW" can mean something different each time it is encountered (GLO less Switzerland in one case, GLO - a set of 50 countries in another, etc.).   

RoWer is used to:  
(1) Generate a list of "specific" RoW regions (RoW-0, RoW_1, etc.) for each RoW encountered in a database;  
(2) Associate with each specific RoW region the list of locations that are *excluded*  
(3) Map activities to these specific RoW regions so we know which to use in regionalized LCA.  

## Usage  
`RoWer` is meant to be used with the [Brightway2 LCA framework](https://brightwaylca.org/).  
It is theoretically useful with any database that includes RoW regions. For now, to our knowledge, only the ecoinvent database does this.  

### Setting up the project

In [1]:
from bw2data import projects, Database
from bw2io import SingleOutputEcospold2Importer, bw2setup

In [3]:
import RoWer

In [4]:
projects.set_current('RoWer generation')
bw2setup()

Creating default biosphere



Writing activities to SQLite3 database:
0%                          100%
[###                           ] | ETA: 00:00:01

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applied 2 strategies in 0.01 seconds


[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:01


Title: Writing activities to SQLite3 database:
  Started: 05/14/2018 23:39:28
  Finished: 05/14/2018 23:39:29
  Total time elapsed: 00:00:01
  CPU %: 46.30
  Memory %: 0.77
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 4 strategies in 1.38 seconds
Wrote 718 LCIA methods with 178008 characterization factors
Creating core data migrations



### ecoinvent 3.4 cutoff by classification

**Import the database**

In [5]:
ei34cutoff_fp = r'C:\mypy\data\ecoinvent_spolds\ecoinvent34\datasets'

In [6]:
ei34cutoff_importer = SingleOutputEcospold2Importer(ei34cutoff_fp, 'ecoinvent 3.4 cutoff')

Extracting XML data from 14889 datasets
Extracted 14889 datasets in 32.87 seconds


In [7]:
ei34cutoff_importer.apply_strategies()
ei34cutoff_importer.statistics()

Applying strategy: normalize_units
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre34
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: set_lognormal_loc_value
Applied 16 strategies in 5.98 seconds
14889 datasets
520205 exchanges
0 unlinked exchanges
  


(14889, 520205, 0)

In [8]:
ei34cutoff_importer.write_database()

Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:02:10


Title: Writing activities to SQLite3 database:
  Started: 05/14/2018 23:42:11
  Finished: 05/14/2018 23:44:22
  Total time elapsed: 00:02:10
  CPU %: 51.10
  Memory %: 6.56
Created database: ecoinvent 3.4 cutoff


Brightway2 SQLiteBackend: ecoinvent 3.4 cutoff

**Generate RoW regions using `RoWer.make_RoWs`**  
This creates two dictionaries:  
- a `RoW definitions` dictionary that identifies what regions are excluded from each specific RoW.  
- an `activity to RoW mapping` dictionary that identifies which RoW are used by activities in the database  

By specifying "modify_db_in_place", the whole database is rewritten with the new RoW definitions.  

In [ ]:
RoW_def, RoW_mapping = RoWer.make_RoWs('ecoinvent 3.4 cutoff', modify_db_in_place=True)

3069 activities with RoW location found in 10.397938966751099 seconds, generating dictionaries
Generated 3069 RoW definitions in 21.905717611312866 seconds
Modifying database ecoinvent 3.4 cutoff in place


Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:02:13


Title: Writing activities to SQLite3 database:
  Started: 05/14/2018 23:46:06
  Finished: 05/14/2018 23:48:19
  Total time elapsed: 00:02:13
  CPU %: 52.30
  Memory %: 16.33


**Write data to disk using RoWer.write_RoW_info**  
The `RoW definitions` and `mapping` dicts can be stored to disk. They are stored in a "Data Package".

In [ ]:
RoWer.write_RoW_info(RoW_def, RoW_mapping, r'C:/mypy/code/RoWer/data')